In [1]:
import glob
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import pandas as pd
from datetime import datetime, timedelta
import calendar
from collections import Counter
import requests
import multiprocessing as mp
from multiprocessing import Pool
import time
from datetime import datetime

# WEB CRAWL
import urllib
import random
import datetime as dt
from itertools import cycle

## Functions

In [4]:
def convert_date(date):
    # Remove special characters
    date = re.sub('[^a-z0-9]', " ", date)
    try:
        # ASSUMING dd MMM
        # Extract day
        day_start = re.search('^\d+', date).start()
        day_end = re.search('^\d+', date).end()
        day = date[day_start:day_end]
        if len(day) == 1:
            day = "0"+day
        # Extract month
        date_wo_day = date[day_end:].strip()
        month_start = re.search('^\w+', date_wo_day).start()
        month_end = re.search('^\w+', date_wo_day).end()
        month = date_wo_day[month_start:month_end]
        if "jan" in str(month):
            month = "01"
        if "feb" in str(month):
            month = "02"
        if "mar" in str(month):
            month = "03"
        if "apr" in str(month):
            month = "04"
        if "may" in str(month):
            month = "05"
        if "jun" in str(month):
            month = "06"
        if "jul" in str(month):
            month = "07"
        if "aug" in str(month):
            month = "08"
        if "sep" in str(month):
            month = "09"
        if "oct" in str(month):
            month = "10"
        if "nov" in str(month):
            month = "11"
        if "dec" in str(month):
            month = "12"

        full_date = day+"."+month+".2019"
        if len(full_date) == 10:
            return datetime.strptime(full_date, '%d.%m.%Y').date()
        else:
            return "nil"
    except:
        return "nil"

In [12]:
def convert_time(time, post_time):
    return_time = "nil"
    time_original = str(re.sub(r'[.:]', '', time)).strip()
    time_original = str(re.sub(r'^([a-zA-Z]+ ){1,}', '', time_original)).strip()
    time_original = str(re.sub(r'\d+\-', '', time_original)).strip()
    
    # ASSUME NUM PM
    if bool(re.search(r'^\d+ ?pm', time_original)):
        time = time_original.replace("pm", "").strip()
        time = str(time)
        # If 1,2,3,4,5,6,7,8,9
        if len(str(time)) == 1:
            time = int(time)
            if time == 1:
                return_time = "13:00"
            if time == 2:
                return_time = "14:00"
            if time == 3:
                return_time = "15:00"
            if time == 4:
                return_time = "16:00"
            if time == 5:
                return_time = "17:00"
            if time == 6:
                return_time = "18:00"
            if time == 7:
                return_time = "19:00"
            if time == 8:
                return_time = "20:00"
            if time == 9:
                return_time = "21:00"
        # If 10,11,12
        if len(str(time)) == 2:
            time = int(time)
            if time == 10:
                return_time = "22:00"
            if time == 11:
                return_time = "23:00"
            if time == 12:
                return_time = "12:00"
        # If single hour (eg. 1) and minutes (eg. 15)
        if len(str(time)) == 3:
            hour = int(time[0])
            if hour == 1:
                return_time = "13:"+time[1:]
            if hour == 2:
                return_time = "14:"+time[1:]
            if hour == 3:
                return_time = "15:"+time[1:]
            if hour == 4:
                return_time = "16:"+time[1:]
            if hour == 5:
                return_time = "17:"+time[1:]
            if hour == 6:
                return_time = "18:"+time[1:]
            if hour == 7:
                return_time = "19:"+time[1:]
            if hour == 8:
                return_time = "20:"+time[1:]
            if hour == 9:
                return_time = "21:"+time[1:]
        # If double hour (eg. 11) and minutes (eg. 30)
        if len(str(time)) == 4:
            hour = int(time[0:2])
            if hour == 10:
                return_time = "22:"+time[2:]
            if hour == 11:
                return_time = "23:"+time[2:]
            if hour == 12:
                return_time = "12:"+time[2:]
    
    # ASSUME NUM AM
    if bool(re.search(r'^\d+ ?am', time_original)):
        time = time_original.replace("am", "").strip()
        time = str(time)
        # If 1,2,3,4,5,6,7,8,9
        if len(str(time)) == 1:
            time = int(time)
            if time == 1:
                return_time = "01:00"
            if time == 2:
                return_time = "02:00"
            if time == 3:
                return_time = "03:00"
            if time == 4:
                return_time = "04:00"
            if time == 5:
                return_time = "05:00"
            if time == 6:
                return_time = "06:00"
            if time == 7:
                return_time = "07:00"
            if time == 8:
                return_time = "08:00"
            if time == 9:
                return_time = "09:00"
        # If 10,11,12
        if len(str(time)) == 2:
            time = int(time)
            if time == 10:
                return_time = "10:00"
            if time == 11:
                return_time = "11:00"
            if time == 12:
                return_time = "00:00"
        # If single hour (eg. 1) and minutes (eg. 15)
        if len(str(time)) == 3:
            hour = int(time[0])
            if hour == 1:
                return_time = "01:"+time[1:]
            if hour == 2:
                return_time = "02:"+time[1:]
            if hour == 3:
                return_time = "03:"+time[1:]
            if hour == 4:
                return_time = "04:"+time[1:]
            if hour == 5:
                return_time = "05:"+time[1:]
            if hour == 6:
                return_time = "06:"+time[1:]
            if hour == 7:
                return_time = "07:"+time[1:]
            if hour == 8:
                return_time = "08:"+time[1:]
            if hour == 9:
                return_time = "09:"+time[1:]
        # If double hour (eg. 11) and minutes (eg. 30)
        if len(str(time)) == 4:
            hour = int(time[0:2])
            if hour == 10:
                return_time = "10:"+time[2:]
            if hour == 11:
                return_time = "11:"+time[2:]
            if hour == 12:
                return_time = "00:"+time[2:]
    
    # ASSUME NO PM OR AM
    if bool(re.search(r'^\d+$', time_original)):
        time = str(time_original).strip()
        # If single digit
        if len(time) == 1:
            time = int(time)
            post_hour = post_time[:2]
            if int(post_hour) >= 12:
                if time == 1:
                    return_time = "13:00"
                if time == 2:
                    return_time = "14:00"
                if time == 3:
                    return_time = "15:00"
                if time == 4:
                    return_time = "16:00"
                if time == 5:
                    return_time = "17:00"
                if time == 6:
                    return_time = "18:00"
                if time == 7:
                    return_time = "19:00"
                if time == 8:
                    return_time = "20:00"
                if time == 9:
                    return_time = "21:00"
            else:
                if time == 1:
                    return_time = "01:00"
                if time == 2:
                    return_time = "02:00"
                if time == 3:
                    return_time = "03:00"
                if time == 4:
                    return_time = "04:00"
                if time == 5:
                    return_time = "05:00"
                if time == 6:
                    return_time = "06:00"
                if time == 7:
                    return_time = "07:00"
                if time == 8:
                    return_time = "08:00"
                if time == 9:
                    return_time = "09:00"
        # If double digits
        if len(str(time)) == 2:
            time = int(time)
            post_hour = post_time[:2]
            if int(post_hour) >= 12:
                if time == 10:
                    return_time = "22:00"
                if time == 11:
                    return_time = "23:00"
                if time == 12:
                    return_time = "12:00"
            else:
                if time == 10:
                    return_time = "10:00"
                if time == 11:
                    return_time = "11:00"
                if time == 12:
                    return_time = "00:00"    
        # If three digtis
        if len(str(time)) == 3:
            hour = int(str(time)[0])
            post_hour = post_time[:2]
            if int(post_hour) >= 12:
                if hour == 1:
                    return_time = "13:"+time[1:]
                if hour == 2:
                    return_time = "14:"+time[1:]
                if hour == 3:
                    return_time = "15:"+time[1:]
                if hour == 4:
                    return_time = "16:"+time[1:]
                if hour == 5:
                    return_time = "17:"+time[1:]
                if hour == 6:
                    return_time = "18:"+time[1:]
                if hour == 7:
                    return_time = "19:"+time[1:]
                if hour == 8:
                    return_time = "20:"+time[1:]
                if hour == 9:
                    return_time = "21:"+time[1:]
            else:
                if hour == 1:
                    return_time = "01:"+time[1:]
                if hour == 2:
                    return_time = "02:"+time[1:]
                if hour == 3:
                    return_time = "03:"+time[1:]
                if hour == 4:
                    return_time = "04:"+time[1:]
                if hour == 5:
                    return_time = "05:"+time[1:]
                if hour == 6:
                    return_time = "06:"+time[1:]
                if hour == 7:
                    return_time = "07:"+time[1:]
                if hour == 8:
                    return_time = "08:"+time[1:]
                if hour == 9:
                    return_time = "09:"+time[1:]
        # If four digtis
        if len(str(time)) == 4:
            hour = int(str(time)[0:2])
            post_hour = post_time[:2]
            if int(post_hour) >= 12:
                if hour >= 12:
                    return_time = str(hour)+":"+str(time)[2:]
                if hour == 10:
                    return_time = "22:"+time[2:]
                if hour == 11:
                    return_time = "23:"+time[2:]
                if hour == 0:
                    return_time = "00:"+time[2:]
            else:
                if hour <= 12:
                    if len(str(hour)) == 1:
                        return_time = "0"+str(hour)+":"+str(time)[2:]
                    else:
                        return_time = str(hour)+":"+str(time)[2:]
                if hour == 0:
                    return_time = "00:"+time[2:]
                if hour == 10:
                    return_time = "10:"+time[2:]
                if hour == 11:
                    return_time = "11:"+time[2:]
                if hour == 12:
                    return_time = "00:"+time[2:]
        
    return return_time

## Process Data

### Read Files

In [6]:
html_files_old = glob.glob("./telegram_chat_export_beginning_19sep/*.html")
print("Number of files: ", len(html_files_old))
print(html_files_old[:10])
print()

html_files_new = glob.glob("./telegram_chat_export_13oct_10dec/*.html")
print("Number of files: ", len(html_files_new))
print(html_files_new[:10])

Number of files:  257
['./telegram_chat_export_beginning_19sep\\messages.html', './telegram_chat_export_beginning_19sep\\messages10.html', './telegram_chat_export_beginning_19sep\\messages100.html', './telegram_chat_export_beginning_19sep\\messages101.html', './telegram_chat_export_beginning_19sep\\messages102.html', './telegram_chat_export_beginning_19sep\\messages103.html', './telegram_chat_export_beginning_19sep\\messages104.html', './telegram_chat_export_beginning_19sep\\messages105.html', './telegram_chat_export_beginning_19sep\\messages106.html', './telegram_chat_export_beginning_19sep\\messages107.html']

Number of files:  160
['./telegram_chat_export_13oct_10dec\\messages.html', './telegram_chat_export_13oct_10dec\\messages10.html', './telegram_chat_export_13oct_10dec\\messages100.html', './telegram_chat_export_13oct_10dec\\messages101.html', './telegram_chat_export_13oct_10dec\\messages102.html', './telegram_chat_export_13oct_10dec\\messages103.html', './telegram_chat_export_1

In [7]:
html_files = [*html_files_old, *html_files_new]
print("Total html files: ", len(html_files))

Total html files:  417


### Extract Data from Files

In [64]:
messages_list = []
total_messages = 0

for file in tqdm(html_files):    
    f = open(file, "r", encoding='utf-8')
    chat = f.read()
    soup = BeautifulSoup(chat)
    messages = soup.findAll("div", {"class": "message default clearfix"})
    total_messages += len(messages)

    for message in messages:
        # Get user
        user = message.find("div", {"class": "from_name"}).text.replace("\n", "").strip()
        #print(user)
        # Get post date and time
        post_datetime = message.find("div", {"class": "pull_right date details"}).get("title")
        post_date = post_datetime.split(" ")[0]
        post_time = post_datetime.split(" ")[1]
        #print(post_date)
        #print(post_time)
        # GET MESSAGE
        raw_message = message.find("div", {"class": "text"})
        # Extract text from message
        raw_message = str(raw_message).replace("</div>", "").replace('<div class="text">', "").replace("<strong>", "").replace("</strong>", "").replace("<u>", "").replace("</u>", "").strip().lower()
        # Tokenize text
        raw_message_tokens = re.sub(r'(<br/> ?){2,}', '<br/>', raw_message).split("<br/>")

        # Variable flag for actual ride request
        is_request = False
        # Get user type
        if(bool(re.search(r'hitchers? looking', raw_message_tokens[0]))):
            user_type = "hitcher"
            is_request = True
        elif(bool(re.search(r'driver? looking', raw_message_tokens[0]))):
            user_type = "driver"
            is_request = True
        else:
            user_type = "nil"

        # PROCESS MESSAGE CONTENT IF IT IS ACTUALLY A REQUEST
        if(is_request):
            # Blank place holder
            pick_up = "nil"
            drop_off = "nil"
            request_date = "nil"
            request_time = "nil"
            number_of_pax = "nil"
            request_date_normalised = "nil"
            request_time_normalised = "nil"
            day = "nil"
            # ------------------

            # Extract pick up location assuming it is always the second token
            try:
                pick_up_tokens = str(raw_message_tokens[1]).split(":")
            except:
                pass
            try:
                if "pick" in pick_up_tokens[0]:
                    pick_up = pick_up_tokens[1].strip()
            except:
                pick_up_tokens = str(raw_message_tokens[1]).split(" ")
                if "pick" in pick_up_tokens[0]:
                    pick_up = " ".join(pick_up_tokens[1:])
                else:
                    pass

            # Extract drop off location assuming it is always the third token
            try:
                drop_off_tokens = str(raw_message_tokens[2]).split(":")
            except:
                pass
            try:
                if "drop" in drop_off_tokens[0]:
                    drop_off = drop_off_tokens[1].strip()
            except:
                drop_off_tokens = str(raw_message_tokens[2]).split(" ")
                if "drop" in drop_off_tokens[0]:
                    drop_off = " ".join(drop_off_tokens[1:])
                else:
                    pass

            # Extract date assuming it is always the fourth token
            try:
                date_tokens = str(raw_message_tokens[3]).split(":")
            except:
                pass
            try:
                if "date" in date_tokens[0]:
                    request_date = date_tokens[1].strip()
            except:
                date_tokens = str(raw_message_tokens[3]).split(" ")
                if "date" in date_tokens[0]:
                    request_date = " ".join(date_tokens[1:])
                else:
                    pass
            if "today" in request_date or "tdy" in request_date:
                request_date = datetime.strptime(post_date, '%d.%m.%Y').date()
            elif "tmr" in request_date or "tomorrow" in request_date:
                today_date = datetime.strptime(post_date, '%d.%m.%Y').date()
                tomorrow_date = today_date + timedelta(days=1)
                request_date = tomorrow_date
            elif request_date == "nil":
                pass
            else:
                request_date = convert_date(request_date)
            
            # Extract time assuming it is always the fifth token
            try:
                time_raw = str(raw_message_tokens[4])
            except:
                pass
            try:
                time = time_raw.replace("time:", "").replace("time :", "").strip()
                time = re.sub(r'[^a-z0-9.-:]', '', time).strip()
                if "now" in time:
                    request_time = post_time[:5]
                else:
                    request_time = convert_time(str(time), post_time)
            except:
                pass
                
            # Extract number of pax assuming it is always the sixth token
            try:
                if "pax" in raw_message_tokens[5]:
                    raw_pax = str(raw_message_tokens[5]).replace("pax:", "").replace("pax", "").replace("pax :", "").replace("pax : ", "").strip()
                    raw_pax = raw_pax.replace(":", "").strip()
                    raw_pax = re.sub(r'\(.*\)', '', raw_pax)
                    raw_pax = re.sub(r'[^a-z0-9- ]', '', raw_pax)
                    raw_pax = raw_pax.strip()
                    if len(raw_pax) == 1:
                        number_of_pax = raw_pax
                    elif len(raw_pax) == 3:
                        number_of_pax = raw_pax
                    else:
                        if bool(re.search(r'^\d+ - \d+$', raw_pax)):
                            number_of_pax = raw_pax.replace(" ", "")
                        if bool(re.search(r'^\d+ to \d+$', raw_pax)):
                            number_of_pax = raw_pax.replace("to", "-")
                            number_of_pax = number_of_pax.replace(" ", "")
            except:
                pass
            
            # If request_date and time is "nil" format of message might be wrong
            if request_date == "nil" or request_time == "nil":
                # Extract time again
                try:
                    time_raw = re.search(r'time\:.*\<br\/\>', raw_message).group()
                    time = time_raw.split("<br/>")[0]
                    time = time.replace("time:", "").strip()  
                    if "now" in time:
                        request_time = post_time[:5]
                    else:
                        request_time = convert_time(str(time), post_time)
                except:
                    pass
                # Extract date again
                try:
                    date_raw = re.search(r'date\:.*\<br\/\>', raw_message).group()
                    request_date = date_raw.split("<br/>")[0]
                    request_date = date.replace("date:", "").strip()
                    if "today" in request_date or "tdy" in request_date:
                        request_date = datetime.strptime(post_date, '%d.%m.%Y').date()
                    elif "tmr" in request_date or "tomorrow" in request_date:
                        today_date = datetime.strptime(post_date, '%d.%m.%Y').date()
                        tomorrow_date = today_date + timedelta(days=1)
                        request_date = tomorrow_date
                    else:
                        request_date = convert_date(request_date)
                except:
                    if request_time != "nil":
                        request_date = datetime.strptime(post_date, '%d.%m.%Y').date()
                # Extract pax again
                try:
                    if bool(re.search(r'pax:.*\<br\/\>', raw_message)):
                        raw_pax = re.search(r'pax:.*\<br\/\>', lol).group()
                        raw_pax = raw_pax.split("<br/>")[0]
                        raw_pax = raw_pax.replace("pax:", "").strip()
                    if bool(re.search(r'pax:.*$', lol)):
                        raw_pax = re.search(r'pax:.*$', lol).group()
                        raw_pax = raw_pax.replace("pax:", "").strip()
                except:
                    pass
            
            #--------------------NORMALISE DATA---------------------
            
            # Normalise date and time
            try:
                date_time = datetime.strptime(str(request_date)+" "+request_time, '%Y-%m-%d %H:%M')
                approx = round(date_time.minute/30.0) * 30
                date_time = date_time.replace(minute=0)
                date_time += timedelta(seconds=approx*60)
                request_date_normalised = date_time.date()
                request_time_normalised = date_time.strftime('%H:%M')
            except:
                pass
            
            # Add day to data eg. dd-mm-yyyy = wed
            try:
                my_date = datetime.strptime(str(request_date_normalised), '%Y-%m-%d')
                day = calendar.day_name[my_date.weekday()].lower()
            except:
                pass
            
            # Store structured message into dictionary
            message_data = {
                'user' : user,
                'post_date' : post_date,
                'post_time' : post_time,
                'raw_message' : raw_message,
                'user_type' : user_type,
                'pick_up' : pick_up,
                'drop_off' : drop_off,
                'request_date' : request_date,
                'request_date_normalised' : request_date_normalised,
                'request_time' : request_time,
                'request_time_normalised' : request_time_normalised,
                'day' : day,
                'number_of_pax' : number_of_pax
            }
            messages_list.append(message_data)

100%|████████████████████████████████████████████████████████████████████████████████| 417/417 [05:49<00:00,  1.09it/s]


### Stats

In [65]:
print("Total number of messages: \t\t", total_messages)
print("Total number of actual messages: \t", len(messages_list))
print()
print("Percentage of actual messages: \t", round((len(messages_list)/total_messages)*100, 2), "%")

Total number of messages: 		 376598
Total number of actual messages: 	 360464

Percentage of actual messages: 	 95.72 %


### Convert Data to DataFrame

In [ ]:
data = pd.DataFrame(messages_list)
data.head()
data.to_csv("./data/sg_hitch_data_raw_updated.csv")

### Filter out Corrupted Data

In [2]:
data = pd.read_csv("./data/sg_hitch_data_raw_updated.csv", index_col=0)

# Filter out corrupted request time
clean_data_rt = data[data.request_time_normalised != "nil"]
print("Removed", len(data)-len(clean_data_rt), "corrupted request time")
# Filter out corrupted request date
clean_data_rd = clean_data_rt[clean_data_rt.request_date_normalised != "nil"]
print("Removed", len(clean_data_rt)-len(clean_data_rd), "corrupted request date")
# Filter out corrupted pick up
clean_data_pu = clean_data_rd[clean_data_rd.pick_up != "nil"]
print("Removed", len(clean_data_rd)-len(clean_data_pu), "corrupted pick up")
# Filter out corrupted drop off
clean_data_do = clean_data_pu[clean_data_pu.drop_off != "nil"]
print("Removed", len(clean_data_pu)-len(clean_data_do), "corrupted drop off")

Removed 18292 corrupted request time
Removed 0 corrupted request date
Removed 318 corrupted pick up
Removed 2341 corrupted drop off


In [3]:
clean_data = clean_data_do
print("Number of requests left: ", len(clean_data))
print()
print("Percentage of non-corrupted data: ", round(len(clean_data)/len(data)*100, 2), "%")

Number of requests left:  339513

Percentage of non-corrupted data:  94.19 %


In [4]:
clean_data.head()

,day,drop_off,number_of_pax,pick_up,post_date,post_time,raw_message,request_date,request_date_normalised,request_time,request_time_normalised,user,user_type
1,tuesday,bedok reservoir,nil,tuas area,26.02.2019,14:55:47,driver looking for hitchers<br/>pick up: tuas ...,2019-02-26,2019-02-26,18:15,18:00,xoxoxo,driver
2,tuesday,off - ntu,nil,up - mountbatten mrt,26.02.2019,15:00:23,hitcher looking for drivers<br/>pick up - moun...,2019-02-26,2019-02-26,17:00,17:00,Amanda,hitcher
3,tuesday,north-east,nil,bedok/tamp/pasir ris/eunos,26.02.2019,15:39:19,driver looking for hitchers<br/>pick up: bedok...,2019-02-26,2019-02-26,16:00,16:00,J,driver
4,tuesday,north-east or east,nil,jurong,26.02.2019,18:12:47,driver looking for hitchers<br/>pick up: juron...,2019-02-26,2019-02-26,18:12,18:00,Colonel Ronald Silver,driver
7,tuesday,suss,nil,jurong west,26.02.2019,23:50:17,hitcher looking for drivers <br/>pick up: juro...,2019-02-26,2019-02-26,07:30,07:30,a. Lee,hitcher


### Normalise Data

In [51]:
# Clean up pick_up and drop_off
def clean_location(loc):
    # Split multiple nearby locations to use first
    if '/' in loc:
        loc = loc.split('/')[0].strip()
    # Replace short forms
    clean = re.sub(r'\,', ' ', loc)
    clean = re.sub(r'  ', ' ', clean)
    clean = re.sub(r'\(.*\)', '', clean)
    clean = re.sub(r'\<.*\>', '', clean)
    clean = re.sub(r'\<|\>', '', clean)
    clean = re.sub(r'\[.*\]', '', clean)
    clean = re.sub(r'--\&gt;', '', clean)
    clean = re.sub(r'\barea\b$', '', clean)
    clean = re.sub(r'\bblock\b', '', clean)
    clean = re.sub(r'\bblk\b', '', clean)
    clean = re.sub(r'\road\b', 'rd', clean)
    clean = re.sub(r'\bavenue\b', 'ave', clean)
    clean = re.sub(r'\bstreet\b', 'st', clean)
    clean = re.sub(r'\bamk\b', 'ang mo kio', clean)
    clean = re.sub(r'\bcck\b', 'choa chu kang', clean)
    clean = re.sub(r'\btpy\b', 'toa payoh', clean)
    clean = re.sub(r'\byck\b', 'yio chu kang', clean)
    clean = re.sub(r'^vivo$', 'vivo city', clean)
    clean = re.sub(r'\btamp\b', 'tampines', clean)
    clean = re.sub(r'\bseng kang\b', 'sengkang', clean)
    clean = re.sub(r'\bcq\b', 'clarke quay', clean)
    clean = re.sub(r'\bwlds\b', 'woodlands', clean)
    clean = re.sub(r'^\bwoodland\b$', 'woodlands', clean)
    clean = re.sub(r'^\bsim\b$', 'singapore institute of management', clean)
    clean = re.sub(r' {2,}', ' ', clean)
    return clean.strip()

pick_up = clean_data['pick_up'].tolist()
drop_off = clean_data['drop_off'].to_list()

pick_up_clean = []
drop_off_clean = []

for pu in tqdm(pick_up):
    pick_up_clean.append(clean_location(str(pu)))
    
for do in tqdm(drop_off):
    drop_off_clean.append(clean_location(str(do)))

100%|██████████| 339513/339513 [00:05<00:00, 67520.93it/s]


In [5]:
print(Counter(pick_up_clean).most_common(20))
print()
print(Counter(drop_off_clean).most_common(20))

[('woodlands', 10057), ('yishun', 8468), ('sengkang', 7876), ('tampines', 7764), ('punggol', 6238), ('hougang', 6020), ('choa chu kang', 5850), ('ang mo kio', 5549), ('pasir ris', 5407), ('jurong west', 4235), ('bedok', 3986), ('jurong', 3864), ('sembawang', 3307), ('bukit batok', 3296), ('bukit panjang', 3025), ('town', 2203), ('clarke quay', 2196), ('toa payoh', 1928), ('serangoon', 1812), ('clementi', 1753)]

[('anywhere', 9975), ('woodlands', 9294), ('yishun', 7988), ('tampines', 7308), ('sengkang', 7119), ('hougang', 6003), ('punggol', 5971), ('choa chu kang', 5686), ('ang mo kio', 5269), ('pasir ris', 4964), ('jurong west', 3978), ('jurong', 3889), ('bedok', 3720), ('town', 3163), ('bukit panjang', 3070), ('sembawang', 2930), ('bukit batok', 2870), ('clementi', 1936), ('west', 1887), ('toa payoh', 1833)]


In [7]:
# Check if values change
for i in range(5004,5010):
    print("Org: ", pick_up[i])
    print("Clean: ", pick_up_clean[i])
    print()

Org:  wlds 731628
Clean:  woodlands 731628

Org:  star vista
Clean:  star vista

Org:  jurong
Clean:  jurong

Org:  760236
Clean:  760236

Org:  jurong east/bukit batok
Clean:  jurong east

Org:  vivo
Clean:  vivo city



In [107]:
# Add columns to dataframe
data_normalised = clean_data.assign(pick_up_clean=pick_up_clean,
                                    drop_off_clean=drop_off_clean)
data_normalised.head()

,day,drop_off,number_of_pax,pick_up,post_date,post_time,raw_message,request_date,request_date_normalised,request_time,request_time_normalised,user,user_type,pick_up_clean,drop_off_clean
1,tuesday,bedok reservoir,nil,tuas area,26.02.2019,14:55:47,driver looking for hitchers<br/>pick up: tuas ...,2019-02-26,2019-02-26,18:15,18:00,xoxoxo,driver,tuas,bedok reservoir
2,tuesday,off - ntu,nil,up - mountbatten mrt,26.02.2019,15:00:23,hitcher looking for drivers<br/>pick up - moun...,2019-02-26,2019-02-26,17:00,17:00,Amanda,hitcher,up - mountbatten mrt,off - ntu
3,tuesday,north-east,nil,bedok/tamp/pasir ris/eunos,26.02.2019,15:39:19,driver looking for hitchers<br/>pick up: bedok...,2019-02-26,2019-02-26,16:00,16:00,J,driver,bedok,north-east
4,tuesday,north-east or east,nil,jurong,26.02.2019,18:12:47,driver looking for hitchers<br/>pick up: juron...,2019-02-26,2019-02-26,18:12,18:00,Colonel Ronald Silver,driver,jurong,north-east or east
7,tuesday,suss,nil,jurong west,26.02.2019,23:50:17,hitcher looking for drivers <br/>pick up: juro...,2019-02-26,2019-02-26,07:30,07:30,a. Lee,hitcher,jurong west,suss


In [7]:
#from fake_useragent import UserAgent
#ua = UserAgent()
#HEADER = {'User-Agent': ua.random}
HEADERS_LIST = [
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; x64; fr; rv:1.9.2.13) Gecko/20101203 Firebird/3.6.13',
    'Mozilla/5.0 (compatible, MSIE 11, Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
    'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16',
    'Mozilla/5.0 (Windows NT 5.2; RW; rv:7.0a1) Gecko/20091211 SeaMonkey/9.23a1pre'
]

PROXY_URL = 'https://free-proxy-list.net/'

def get_proxies():
    response = requests.get(PROXY_URL)
    soup = BeautifulSoup(response.text, 'lxml')
    table = soup.find('table',id='proxylisttable')
    list_tr = table.find_all('tr')
    list_td = [elem.find_all('td') for elem in list_tr]
    list_td = list(filter(None, list_td))
    list_ip = [elem[0].text for elem in list_td]
    list_ports = [elem[1].text for elem in list_td]
    list_proxies = [':'.join(elem) for elem in list(zip(list_ip, list_ports))]
    return list_proxies

In [8]:
# Get lat-long of location pick_up_clean and drop_off_clean
def get_address(row_index):
    time.sleep(0.1)
    #-----PARAMETERS------
    HEADER = {'User-Agent': random.choice(HEADERS_LIST)}
    proxy = next(proxy_pool)
    #---------------------
    
    pick_up_geo = "nil"
    drop_off_geo = "nil"
    
    if bool(re.search(r'\d{6}', data_normalised.iloc[row_index]['pick_up_clean'])):
        pu_location = re.search(r'\d{6}', data_normalised.iloc[row_index]['pick_up_clean']).group()
    else:
        pu_location = data_normalised.iloc[row_index]['pick_up_clean']
    onemap_url = "https://developers.onemap.sg/commonapi/search"
    onemap_params = {'searchVal' : pu_location, 'returnGeom' : "Y", 'getAddrDetails' : "Y"}
    r = requests.get(url = onemap_url, params = onemap_params, headers=HEADER, proxies={"http": proxy}, timeout=60)
    data = r.json()
    try:
        lat = data['results'][0]['LATITUDE']
        long = data['results'][0]['LONGTITUDE']
        lat_long = lat+","+long
        pick_up_geo = lat_long
    except:
        pass
    
    if bool(re.search(r'\d{6}', data_normalised.iloc[row_index]['drop_off_clean'])):
        do_location = re.search(r'\d{6}', data_normalised.iloc[row_index]['drop_off_clean']).group()
    else:
        do_location = data_normalised.iloc[row_index]['drop_off_clean']
    onemap_params = {'searchVal' : do_location, 'returnGeom' : "Y", 'getAddrDetails' : "Y"}
    r = requests.get(url = onemap_url, params = onemap_params, headers=HEADER, proxies={"http": proxy}, timeout=60)
    data = r.json()
    try:
        lat = data['results'][0]['LATITUDE']
        long = data['results'][0]['LONGTITUDE']
        lat_long = lat+","+long
        drop_off_geo = lat_long
    except:
        pass
    
    result_string = str(row_index)+","+pu_location+","+pick_up_geo+","+do_location+","+drop_off_geo
    
    f = open("./data/geoloc.txt", "a")
    f.write("%s\n" %result_string)
    print(result_string)
    return pd.DataFrame([[row_index, pu_location, pick_up_geo, do_location, drop_off_geo]], columns=['index', 'pick_up_loc', 'pick_up_geo', 'drop_off_loc', 'drop_off_geo'])

In [73]:
proxies = get_proxies()
proxy_pool = cycle(proxies)

start = datetime.now()
with Pool(15) as p:
    result = p.map(get_address, range(len(data_normalised)-1))
print("time: ", datetime.now()-start)

data_geocode = pd.concat(result)
#data_geocode.to_csv("./data/sg_hitch_data_geocode.csv")
#data_geocode.head()

300001,mbs,1.2839941013920602,103.85945063691901,bukit batok,1.36053358595052,103.74236455589701
303412,310201,1.3411380360779699,103.850126597142,554910,1.37446629705358,103.869082561303
302993,mbs,1.2839941013920602,103.85945063691901,panjang,1.2794825436248,103.785805848029
300419,lavender,1.3428954003190499,103.69689412475799,bukit merah,1.28350307111324,103.823353726932
302784,somerset,1.30003194313605,103.83838564393601,clementi,1.2963298251589699,103.7676022071
303830,212672,1.30903703924736,103.852847667323,762813,1.41738877559453,103.83236873739
304249,737897,1.4323447810453502,103.79794676826799,ngee ann poly 51,1.3321583838439601,103.774417260562
300210,126c kim tian road,1.28231370027501,103.82931150361401,10m braddell hill,1.3403521290315301,103.84119927243201
300838,31a brighton ave,1.3597617141751999,103.873196402078,10 heng mui keng terrace,1.291820330256,103.77447663647301
304040,111 northwave woodlands,1.4437374355444499,103.810309503009,640707,1.34178304696078,103.69

300633,tampines ave 5 147,1.34923238819561,103.942181032929,changi village,1.38870594064707,103.987980185347
300215,112 katong,1.30508681845447,103.905098915055,bedok,1.32368201682055,103.947789319297
300843,zouk,nil,ang mo kio or hougang,nil
303626,clementi,1.2963298251589699,103.7676022071,woodlands,1.41520591465648,103.758554325238
302789,bukit batok,1.3508238085532098,103.73826825406601,anywhere,nil
300007,sengkang,1.38315380470819,103.89238167637801,jurong,1.34390440884759,103.766254087753
303418,hougang ave 3,1.3589132090336697,103.892703963042,hougang ave 8,1.37790040917927,103.894019367541
301262,d'underground,nil,670502,1.3837172129332802,103.76675082813802
303836,jurong,1.34390440884759,103.766254087753,hougang,1.3799438359273999,103.887465509279
301053,woodlands,1.41520591465648,103.758554325238,town,1.30923241301234,103.842649053422
304255,choa chu kang,1.4038469560746203,103.749878668123,bugis,1.29950761348254,103.854370864874
303208,punggol place,1.40305080910265,103.9029

303424,yew tee,1.3973210291170202,103.753758637401,ngee ann poly,1.33248057179978,103.77330588052901
302795,520872,1.35422172661635,103.932354147638,730730,1.44253330206837,103.797706231324
300639,anchorvale,1.39635987610917,103.889317071957,sengkang,1.38315380470819,103.89238167637801
301268,d underground,nil,680602,1.39724418317227,103.744501292642
303632,wodlands,nil,yishun,1.41694678230154,103.845000884876
304052,670167,1.37768707307102,103.76510240158899,550518,1.3716411815519398,103.874103124013
303842,golden mile,1.30277120504471,103.865172134495,teck whye,1.38012990422533,103.758203987241
300013,woodlands,1.41520591465648,103.758554325238,530554,1.3787433056641698,103.889681354986
300430,lavender,1.3428954003190499,103.69689412475799,bukit merah,1.28350307111324,103.823353726932
303214,woodlands dr 14,1.43330796704041,103.789299864497,hotel jen tanglin,1.30427053823356,103.82391235958099
301059,wan singapore,1.38380152779493,103.749890683004,woodlands,1.41520591465648,103.75855

304266,hougang,1.3799438359273999,103.887465509279,hitachi square,1.35571330189515,103.941302004065
303219,woodlands dr 14,1.43330796704041,103.789299864497,hotel jen tanglin,1.30427053823356,103.82391235958099
304058,238869,1.306078557128,103.831014834847,575937,1.3753726445818302,103.83204110893499
303638,canberra road,1.4523950974988702,103.81998008454599,singapore institute of management,1.32913701690828,103.776185827202
304267,choa chu kang,1.4038469560746203,103.749878668123,asia square tower 2,1.2783072963325401,103.85079518087099
300227,112 katong,1.30508681845447,103.905098915055,bedok,1.32368201682055,103.947789319297
303011,love drunk princep,nil,bedok reservoir,1.34231030001047,103.925857758506
300855,pasir ris,1.3799565415449098,103.96361889803002,anywhere,nil
302802,520872,1.35422172661635,103.932354147638,730730,1.44253330206837,103.797706231324
303430,hougang ave 2,1.36706142659908,103.888077255516,suntec tw3,nil
301275,orchard plaza,1.30126533034761,103.84115177072701,

300860,hougang,1.3799438359273999,103.887465509279,ang mo kio,1.3687783887791,103.840273944362
302808,south bridge rd,1.28683466874905,103.847766947664,punggol,1.40546002750829,103.898354203231
303854,plaza sing,1.37976754317117,103.93620756732099,560110,1.3704593701089098,103.837419259859
303644,east side,nil,anywhere,nil
300441,hougang,1.3799438359273999,103.887465509279,pioneer,1.3085131714795701,103.674623963707
303017,bukit panjang,1.3775072762462,103.77360557644,ang mo kio,1.3687783887791,103.840273944362
304273,choa chu kang,1.4038469560746203,103.749878668123,funan,1.29134759697797,103.84998978981301
300652,jem,1.33305451930724,103.743433746954,bukit batok,1.3508238085532098,103.73826825406601
300025,upper thomson,1.39758106126826,103.817856949947,anywhere,nil
301281,bar rouge swissotel,nil,219679,1.3131945280372601,103.847445951243
303435,bishan mrt,1.35057959895608,103.84830499183,ssdc,nil
304064,ang mo kio,1.3687783887791,103.840273944362,balestier,1.32700591692146,103.84589

...

313279,tampines,1.35022823751066,103.928412800091,geylang,1.31289336228688,103.887634992935
314320,831 tampines st 83,1.35036280478376,103.934554541156,441a bukit batok west ave 8,1.3561054854113301,103.739002965384
314944,scape,1.30086543586584,103.83570084035699,bishan,1.3596538482492302,103.832637290371
314735,380015,1.3158732490876999,103.87129618518298,530137,1.3490532162175801,103.88664017294
312238,yishun,1.41694678230154,103.845000884876,tampines,1.35022823751066,103.928412800091
312652,sia training centre,1.3458161546681202,103.965842755849,hougang,1.3799438359273999,103.887465509279
313696,sengkang,1.38315380470819,103.89238167637801,bugis,1.29950761348254,103.854370864874
314526,tampines,1.35022823751066,103.928412800091,changi village,1.38870594064707,103.987980185347
312861,75042,1.45299061364994,103.818532777338,730344,1.43207266699218,103.780455054811
313486,punggol safra,1.41011328769963,103.906031923547,simei,1.337313178397,103.95136942172701
313067,tampines,1.35022823

312244,jurong west,1.34160326621595,103.70808518963301,chinatown,1.2847629885034,103.844469740518
313904,whampoa,1.32155040262605,103.854634897341,siglap rd,1.3202535363947,103.920512885209
314741,mbs,1.2839941013920602,103.85945063691901,upper serangoon crescent,1.3771040903616203,103.90028855798501
314950,nex,1.3506755212032,103.871861615443,somerset,1.30003194313605,103.83838564393601
313702,1a kent ridge rd,1.29131750918748,103.78085796658002,530167,1.35181593598796,103.888448131093
314326,10a boon tiong road,1.28635847107696,103.82962615570999,choa chu kang,1.4038469560746203,103.749878668123
313285,punggol,1.40546002750829,103.898354203231,jurong,1.34390440884759,103.766254087753
314116,woodlands,1.41520591465648,103.758554325238,town,1.30923241301234,103.842649053422
312450,792443,1.3911838045873302,103.873946079101,828836,1.3930427045120701,103.91740135778299
312658,ntu,1.34616863877198,103.679965382245,bukit panjang,1.3775072762462,103.77360557644
312867,ang mo kio,1.368778388

313079,bugis,1.29950761348254,103.854370864874,toa payoh,1.33904144822194,103.857955654132
314332,duxton,1.27703473451393,103.84126103489099,lucky plaza,1.30446378974741,103.834005158524
312456,choa chu kang,1.4038469560746203,103.749878668123,tiong bahru plaza,1.28647119992725,103.827158267789
312251,kaki bukit,1.33599225672815,103.901440431972,jurong west,1.34160326621595,103.70808518963301
314957,mei hwan crescent,1.3534416239383602,103.86037381649099,balestier,1.32700591692146,103.84589530196399
313291,bukit panjang,1.3775072762462,103.77360557644,clarke,1.2894069220289,103.847114861345
313708,tanah merah ferry terminal,1.3122633998891,103.988211916284,punggol,1.40546002750829,103.898354203231
314748,robertson walk,1.29186931810741,103.841454882767,woodlands 11,1.43488097187659,103.802836665616
312873,choa chu kang,1.4038469560746203,103.749878668123,punggol,1.40546002750829,103.898354203231
313498,marina mandarin hotel lobby,nil,serangoon garden my village,nil
313911,whampoa,1.321

314963,punggol,1.40546002750829,103.898354203231,sengkang,1.38315380470819,103.89238167637801
313714,370028,1.33048096275077,103.88657920576401,520277,1.3472781964137501,103.95119351710501
313504,pasir ris mrt,1.37284196938315,103.94933342667,toa payoh lor 1 &amp; 8,nil
312258,sim hq,nil,tampines st 21,1.35340749851579,103.95220972637199
312463,22 sin ming road,1.35746679660694,103.839106803265,causeway point,1.43608867729913,103.78600997367799
313297,yishun,1.41694678230154,103.845000884876,anywhere,nil
314544,cuppage,1.3021555073451,103.840293633443,choa chu kang,1.4038469560746203,103.749878668123
313917,jurong,1.34390440884759,103.766254087753,tiong bahru,1.28188236723141,103.830663094413
314755,west coast,1.29484588174167,103.766282088026,nus,1.30698270981494,103.770671662772
314337,town,1.30923241301234,103.842649053422,choa chu kang,1.4038469560746203,103.749878668123
312879,asia square tower 1,1.2786922741422901,103.85135237789,731624,1.43433582747729,103.799667122109
313084,72

313511,521164,1.3496215142655,103.946000691097,esplanade,1.29312572848339,103.855564111142
312677,nuh,1.29619869406283,103.78253770949699,choa chu kang,1.4038469560746203,103.749878668123
313303,hougang,1.3799438359273999,103.887465509279,east,1.3108349867393,103.92171580265999
313923,bukit timah plaza,1.3386316527745201,103.778585430975,687c woodlandsdr 75,nil
313090,650369,1.3600294598366398,103.751072294962,640737,1.34674600665674,103.70115988016799
313720,keat hong camp,1.37267581722658,103.71625680583999,clementi,1.2963298251589699,103.7676022071
314969,scape,1.30086543586584,103.83570084035699,bishan,1.3596538482492302,103.832637290371
314551,csc changi 2,1.39041093994212,103.976466378722,wan suntec,nil
314762,sengkang,1.38315380470819,103.89238167637801,peaches,nil
312886,85 geylang bahru,1.3195646272053998,103.87371392913,822231,1.40212842361403,103.891466927723
312678,clementi,1.2963298251589699,103.7676022071,changi,1.36314165272807,103.97249095319599
313091,geylang,1.3128933

In [95]:
# Combine data with geo location
geo_loc = pd.read_csv("./data/geo_location.csv")
geo_loc.head()

,row_index,pick_up_location,pu_lat,pu_long,drop_off_location,do_lat,do_long
0,0,tuas,1.31902264002549,103.636565831715,bedok reservoir,1.34231030001047,103.925857758506
1,1,up - mountbatten mrt,nil,nil,off - ntu,nil,nil
2,2,bedok,1.32368201682055,103.947789319297,north-east,nil,nil
3,3,jurong,1.34390440884759,103.766254087753,north-east or east,nil,nil
4,4,jurong west,1.34160326621595,103.708085189633,suss,1.31364076815464,103.790065521402


In [96]:
data_normalised.head()

,day,drop_off,number_of_pax,pick_up,post_date,post_time,raw_message,request_date,request_date_normalised,request_time,request_time_normalised,user,user_type,pick_up_clean,drop_off_clean
1,tuesday,bedok reservoir,nil,tuas area,26.02.2019,14:55:47,driver looking for hitchers<br/>pick up: tuas ...,2019-02-26,2019-02-26,18:15,18:00,xoxoxo,driver,tuas,bedok reservoir
2,tuesday,off - ntu,nil,up - mountbatten mrt,26.02.2019,15:00:23,hitcher looking for drivers<br/>pick up - moun...,2019-02-26,2019-02-26,17:00,17:00,Amanda,hitcher,up - mountbatten mrt,off - ntu
3,tuesday,north-east,nil,bedok/tamp/pasir ris/eunos,26.02.2019,15:39:19,driver looking for hitchers<br/>pick up: bedok...,2019-02-26,2019-02-26,16:00,16:00,J,driver,bedok,north-east
4,tuesday,north-east or east,nil,jurong,26.02.2019,18:12:47,driver looking for hitchers<br/>pick up: juron...,2019-02-26,2019-02-26,18:12,18:00,Colonel Ronald Silver,driver,jurong,north-east or east
7,tuesday,suss,nil,jurong west,26.02.2019,23:50:17,hitcher looking for drivers <br/>pick up: juro...,2019-02-26,2019-02-26,07:30,07:30,a. Lee,hitcher,jurong west,suss


In [104]:
pu_lat = geo_loc[' pu_lat'].tolist()
pu_long = geo_loc[' pu_long'].tolist()
do_lat = geo_loc[' do_lat'].tolist()
do_long = geo_loc[' do_long'].tolist()

In [106]:
data_final = data_normalised.assign(pu_lat=pu_lat,
                                   pu_long=pu_long,
                                   do_lat=do_lat,
                                   do_long=do_long)
data_final.head()

ValueError: Length of values does not match length of index